In [1]:
import pandas as pd
import numpy as np

In [2]:
%pwd

'/home/ian/code/github/utils1/0examples/DL'

In [3]:
df = pd.read_excel('/home/ian/code/github/data/wdstaffs.xlsx')
df.head()

,姓名,工号,头像,头像下载地址示例
0,徐俊,623,c1a6a93a-e846-41df-b8ad-e63eb4db4b49.jpg,https://linkup.wondersgroup.com/avatar/u/b/c1a...
1,潘宁,744,ce66ed4d-bdb2-40d5-84a6-3d91358962ca.jpg,NaN
2,宋元成,1096,592c920c-3a68-4210-bbd0-8c1dcfe95ae6.jpg,NaN
3,吴杰,1294,1a1c6343-c338-4404-9883-e65fb5bc3618.jpg,NaN
4,张丽,1325,bbfc2d8c-c7ca-4b99-94da-20f9997f188b.jpg,NaN


In [4]:
df.shape

(634, 4)

In [5]:
import requests
import os

def download_image(line_index, imageid, filename,fileformat,error_list):
    url = 'https://linkup.wondersgroup.com/avatar/u/b/{}'.format(imageid)
#     print('Downloading', url)
    if imageid.endswith('.png'):
        print('3-------------', filename)
        error_list.append(line_index)
        return 
    try:
        response = requests.get(url)
        if response.status_code == 200:
            save_image(response.content, '/home/ian/code/github/data/staff_images', filename,fileformat,error_list)
        else:
            print('0-------------', filename)
            error_list.append(line_index)
    except:
        print('1-------------', filename)
        error_list.append(line_index)
def save_image(content, imagedir, filename,fileformat,error_list):
    '''
    fileformat: 文件格式，如jpg
    '''
    file_path = '{0}/{1}.{2}'.format(imagedir, filename, fileformat)
    if not os.path.exists(imagedir):
        os.makedirs(imagedir)
#     print(file_path)
    if not os.path.exists(file_path):
        with open(file_path, 'wb') as f:
            f.write(content)
            f.close()
    else:
        error_list.append(line_index)
        print('2-------------', filename)

In [7]:
# error_list = list()
# for line in df.itertuples():
#     download_image(line[0],line[3], '{}_{}'.format(line[2], line[1]),'jpg',error_list)

# len(error_list)



import pickle

# def dump2file(o, file, mode='wb'):
#     with open(file, mode) as f:
#         pickle.dump(o,f)

# dump2file(error_list, 'error_list.pkl')
def loadfromfile(file, mode='rb'):
    with open(file, mode) as f:
        return pickle.load(f)
error_list = loadfromfile('error_list.pkl')
df1 = df.drop(error_list)
df1.shape

(525, 4)

## 人脸匹配

In [8]:
# -*- coding: utf-8 -*-
# 自动识别人脸特征
# filename : find_facial_features_in_picture.py

# 导入pil模块 ，可用命令安装 apt-get install python-Imaging
from PIL import Image, ImageDraw
# 导入face_recogntion模块，可用命令安装 pip install face_recognition
import face_recognition

## 提取所有的人脸编码

In [10]:
count = 0
error_list1 = list()
for line in df1.itertuples():
    #将jpg文件加载到numpy数组中
    try:
        face_image = face_recognition.load_image_file('/home/ian/code/github/data/staff_images/{}_{}.jpg'.format(line[2], line[1]))
        face_encoding = face_recognition.face_encodings(face_image)[0]
    #     print(face_encoding)
#         face_pool[count] = face_encoding
#         count += 1
    #     break
    except:
        print('{}_{}.jpg'.format(line[2], line[1]))
        error_list1.append(line[0])
#         break

3111_陶磊.jpg
3115_陈骏涛.jpg
3235_曹彦起.jpg
4323_张浩.jpg
6206_王野.jpg
6983_王小康.jpg
7762_荣亚茹.jpg
7767_徐涵涵.jpg
8049_胡成燕.jpg
8178_韩卫娟.jpg
8201_黄孝海.jpg
BJ292_武文厅.jpg
BJ305_李光伟.jpg
BJ559_周典财.jpg
BJ630_曹永庆.jpg
CD403_徐雷.jpg
cd411_范伟.jpg
CD541_任红.jpg
CD542_孔晓慧.jpg
HZ367_孙文杰.jpg
KG0016_陈强.jpg
kg117_郭吉浩.jpg
NC84_马宇轩.jpg
nj313_席文.jpg
sx143_李鑫.jpg
SZ216_李德棠.jpg
WH008_杨炎良.jpg


In [13]:
df2 = df1.drop(error_list1)
df2.shape

(498, 4)

In [18]:
face_pool = np.zeros((df2.shape[0], 128))

In [19]:
count = 0
error_list2 = list()
for line in df2.itertuples():
    #将jpg文件加载到numpy数组中
    try:
        face_image = face_recognition.load_image_file('/home/ian/code/github/data/staff_images/{}_{}.jpg'.format(line[2], line[1]))
        face_encoding = face_recognition.face_encodings(face_image)[0]
#         print(face_encoding)
        face_pool[count] = face_encoding
        count += 1
    #     break
    except:
        print('{}_{}.jpg'.format(line[2], line[1]))
        error_list2.append(line[0])
#         break

In [17]:
len(error_list2)

1

In [21]:
face_pool.shape

(498, 128)

In [22]:
from sklearn.metrics.pairwise import pairwise_distances
face_similarity = pairwise_distances(face_pool, metric='cosine')

In [81]:
face_similarity

array([[0.        , 0.09249731, 0.0847767 , ..., 0.09281166, 0.15703254,
        0.09901702],
       [0.09249731, 0.        , 0.10941001, ..., 0.07749533, 0.12556182,
        0.08814324],
       [0.0847767 , 0.10941001, 0.        , ..., 0.06896619, 0.14365868,
        0.07224795],
       ...,
       [0.09281166, 0.07749533, 0.06896619, ..., 0.        , 0.12097428,
        0.07588172],
       [0.15703254, 0.12556182, 0.14365868, ..., 0.12097428, 0.        ,
        0.11053873],
       [0.09901702, 0.08814324, 0.07224795, ..., 0.07588172, 0.11053873,
        0.        ]])

In [26]:
np.argmax(face_similarity[0])

43

In [80]:
np.delete(face_similarity[0],0).shape

(497,)

In [77]:
def find(face_pool, num=None, name=None):
    '''
    找出最相似的员工
    '''
    i = -1
    if name:
        t1 = df2[df2['姓名']==name].index.values
        if len(t1)>0:
            i = t1[0]
        else:
            print('员工 {} 不存在'.format(name))
            return
    elif num:
        t1 = df2[df2['工号']==num].index.values
        if len(t1)>0:
            i = t1[0]
        else:
            print('工号 {} 不存在'.format(name))
            return
    if i == -1:
        print('wrong')
        return
    a = np.delete(face_similarity[i],i)
    ii = np.argmin(a)
    
    iii = ii
    if ii >= i:
        iii += 1
    print('和{}_{}最相似的人是{}_{}，相似度为{}'.format(df2.iloc[i,1],df2.iloc[i,0],df2.iloc[iii,1],df2.iloc[iii,0],1-a[ii]))    
    return ii, a[ii]

In [78]:
for line in df2.iloc[0:].itertuples():
    print(line[2])
    find(face_pool, num=line[2])
#     break

623
和623_徐俊最相似的人是heb34_姜海宇，相似度为0.9491168119085399
744
和744_潘宁最相似的人是1446_杨永飞，相似度为0.9647019678274563
1096
和1096_宋元成最相似的人是4549_杨晓青，相似度为0.9583242354600512
1325
和1325_张丽最相似的人是NC30_宋卓，相似度为0.9488005252096043
1333
和1333_张邕芬最相似的人是CS98_徐孟森，相似度为0.9357250532591203
1446
和1446_杨永飞最相似的人是744_潘宁，相似度为0.9647019678274563
1453
和1453_葛素坤最相似的人是2512_沈佳彬，相似度为0.9513038821207632
1495
和1495_赵斌最相似的人是WH132_董辉，相似度为0.9665362919728936
1565
和1565_徐文华最相似的人是NC108_吴雪兵，相似度为0.9602134647310706
1584
和1584_李莉丽最相似的人是NJ140_徐聿尧，相似度为0.9588097720544486
1635
和1635_孙婷最相似的人是6985_姚媛，相似度为0.956369558666406
1717
和1717_黄明月最相似的人是WH141_陈俊红，相似度为0.9552162496979646
1821
和1821_王奇最相似的人是NB80_万松望，相似度为0.9647738417950436
1906
和1906_狄冬囚最相似的人是BJ558_王宇，相似度为0.9583917987039616
1907
和1907_龚飞最相似的人是NC111_邓浩霖，相似度为0.9513828620122097
1989
和1989_郭瑾最相似的人是5359_严岚，相似度为0.9670009456113267
2075
和2075_张春杰最相似的人是sx55_白冰，相似度为0.9603545895633403
2152
和2152_钱涌最相似的人是sx55_白冰，相似度为0.9590477482234773
2295
和2295_汤士伟最相似的人是NC28_张英斌，相似度为0.9692814442500547
2323
和2323_毛志军最相似的人是6688_周之奇

和NJ347_孙欢最相似的人是WH131_郭珊，相似度为0.9651534555425553
nj358
和nj358_董浩然最相似的人是7695_叶智勇，相似度为0.9605834406143503
nj360
和nj360_祝全业最相似的人是NC31_熊贤焜，相似度为0.9633697619890876
nj361
和nj361_费杰最相似的人是7911_辛鹏飞，相似度为0.9715490359479553
nj362
和nj362_朱留虎最相似的人是6028_张文华，相似度为0.9651620401013848
NJ58
和NJ58_张志航最相似的人是2752_陆鹏飞，相似度为0.9404747474952442
NJ76
和NJ76_崔静圭最相似的人是SZ214_刘明伟，相似度为0.9579009128817785
NJ8
和NJ8_徐东园最相似的人是5941_桑伟毅，相似度为0.9628869607632926
NMG4
和NMG4_訾宇波最相似的人是6986_吕海洋，相似度为0.9716629380347832
NMG5
和NMG5_李泽豪最相似的人是NJ249_袁金鑫，相似度为0.9660181861993333
NX20
和NX20_张鹏最相似的人是7223_马漪伦，相似度为0.9681613222755161
QD128
和QD128_史江维最相似的人是HZ314_王樑，相似度为0.9554969546593572
qd130
和qd130_王磊最相似的人是SX103_王耀民，相似度为0.9592254168525485
QD169
和QD169_李江磊最相似的人是NJ179_刘驰，相似度为0.9706361827681675
qd191
和qd191_田洪兵最相似的人是WH132_董辉，相似度为0.9686531381737816
SD21
和SD21_徐杰最相似的人是5535_虞雅琴，相似度为0.9539029018066395
SD62
和SD62_张路路最相似的人是7316_罗永贵，相似度为0.9475852677687318
SD63
和SD63_张坤最相似的人是sx55_白冰，相似度为0.9577736356661917
SD70
和SD70_黄敏敏最相似的人是SX73_茹蒙鸽，相似度为0.9677213023502069
SD71
和

In [104]:
find(face_pool, name='姚媛')

和6985_姚媛最相似的人是WH90_李柳，相似度为0.9647625497308665


(476, 0.0352374502691335)

In [103]:
find(face_pool, num=3246)

和3246_符鸣最相似的人是2457_韦法林，相似度为0.9378974383048554


(21, 0.06210256169514461)

In [127]:
def new_image(image_path):
    print(image_path)
    image = face_recognition.load_image_file(image_path)
    face_encoding = face_recognition.face_encodings(image)[0]
    tmp_arr = np.vstack((face_pool, face_encoding))
    face_similarity = pairwise_distances(tmp_arr, metric='cosine')
    a = np.delete(face_similarity[-1],-1)
#     print(a)
    ii = np.argmin(a)
    print('最相似的人是{}_{}，相似度为{}'.format(df2.iloc[ii,1],df2.iloc[ii,0],1-a[ii]))    
    return ii, 1-a[ii]

In [128]:
new_image('me.jpg')

me.jpg
最相似的人是7316_罗永贵，相似度为0.9676061651400669


(144, 0.03239383485993308)

In [129]:
new_image('new.jpg')

new.jpg
最相似的人是7419_王娟，相似度为0.9680854307056687


(154, 0.03191456929433134)

In [131]:
new_image('yaoyuan.png')

yaoyuan.png
最相似的人是6985_姚媛，相似度为0.9674790239028055


(125, 0.03252097609719451)

In [97]:
# 将jpg文件加载到numpy 数组中
image = face_recognition.load_image_file('/home/ian/code/github/data/staff_images/7419_王娟.jpg')

#查找图像中所有面部的所有面部特征
face_landmarks_list = face_recognition.face_landmarks(image)

print("I found {} face(s) in this photograph.".format(len(face_landmarks_list)))

I found 1 face(s) in this photograph.


In [98]:
for face_landmarks in face_landmarks_list:

   #打印此图像中每个面部特征的位置
    facial_features = [
        'chin',
        'left_eyebrow',
        'right_eyebrow',
        'nose_bridge',
        'nose_tip',
        'left_eye',
        'right_eye',
        'top_lip',
        'bottom_lip'
    ]

    for facial_feature in facial_features:
        print("The {} in this face has the following points: {}".format(facial_feature, face_landmarks[facial_feature]))
   #让我们在图像中描绘出每个人脸特征！
    pil_image = Image.fromarray(image)
    d = ImageDraw.Draw(pil_image)

    for facial_feature in facial_features:
        d.line(face_landmarks[facial_feature], width=5)

    pil_image.save('tt.jpg')

The chin in this face has the following points: [(111, 84), (111, 97), (112, 111), (114, 123), (120, 135), (128, 144), (137, 153), (148, 159), (159, 161), (169, 159), (177, 152), (184, 145), (189, 136), (193, 126), (196, 116), (198, 106), (198, 96)]
The left_eyebrow in this face has the following points: [(132, 77), (139, 73), (148, 72), (157, 74), (165, 78)]
The right_eyebrow in this face has the following points: [(178, 80), (184, 78), (190, 78), (195, 80), (197, 85)]
The nose_bridge in this face has the following points: [(171, 90), (172, 97), (172, 105), (172, 113)]
The nose_tip in this face has the following points: [(160, 118), (165, 119), (169, 121), (173, 120), (176, 120)]
The left_eye in this face has the following points: [(141, 86), (147, 85), (152, 86), (156, 90), (152, 89), (146, 88)]
The right_eye in this face has the following points: [(179, 93), (184, 90), (188, 91), (191, 93), (188, 94), (184, 94)]
The top_lip in this face has the following points: [(150, 130), (157, 1

In [100]:
face_locations = face_recognition.face_locations(image)
# 打印：我从图片中找到了 多少 张人脸
print("I found {} face(s) in this photograph.".format(len(face_locations)))
# 循环找到的所有人脸
for face_location in face_locations:

        # 打印每张脸的位置信息
        top, right, bottom, left = face_location
        print("A face is located at pixel location Top: {}, Left: {}, Bottom: {}, Right: {}".format(top, left, bottom, right)) 
# 指定人脸的位置信息，然后显示人脸图片
        face_image = image[top:bottom, left:right]
        pil_image = Image.fromarray(face_image)
        pil_image.save('tt1.jpg')

I found 1 face(s) in this photograph.
A face is located at pixel location Top: 56, Left: 116, Bottom: 146, Right: 205


In [89]:
# -*- coding: utf-8 -*-
# 识别人脸鉴定是哪个人

# 导入face_recogntion模块，可用命令安装 pip install face_recognition
import face_recognition

#将jpg文件加载到numpy数组中
image = face_recognition.load_image_file('/home/ian/code/github/data/staff_images/3244_吴昌宝.jpg')
#要识别的图片
unknown_image = face_recognition.load_image_file('/home/ian/code/github/data/staff_images/7419_王娟.jpg')

#获取每个图像文件中每个面部的面部编码
#由于每个图像中可能有多个面，所以返回一个编码列表。
#但是由于我知道每个图像只有一个脸，我只关心每个图像中的第一个编码，所以我取索引0。
chen_face_encoding = face_recognition.face_encodings(chen_image)[0]
unknown_face_encoding = face_recognition.face_encodings(unknown_image)[0]
known_faces = [
    chen_face_encoding
]
#结果是True/false的数组，未知面孔known_faces阵列中的任何人相匹配的结果
results = face_recognition.compare_faces(known_faces, unknown_face_encoding, tolerance=0.6)

print("result :{}".format(results))
print("这个未知面孔是 范冰冰 吗? {}".format(results[0]))
print("这个未知面孔是 我们从未见过的新面孔吗? {}".format(not True in results)) 

result :[False]
这个未知面孔是 范冰冰 吗? False
这个未知面孔是 我们从未见过的新面孔吗? True


In [137]:
np.save('face_pool_498.npy',face_pool)

In [138]:
df2.to_csv('face_wd_498.csv')

In [132]:
# 摄像头头像识别
import face_recognition
import cv2

video_capture = cv2.VideoCapture(0)

In [134]:
# 脸部特征数据的集合
known_face_encodings = face_pool
# 人物名称的集合
known_face_names = list(df2['姓名'])

In [135]:

face_locations = []
face_encodings = []
face_names = []
process_this_frame = True

In [136]:
while True:
    # 读取摄像头画面
    ret, frame = video_capture.read()

    # 改变摄像头图像的大小，图像小，所做的计算就少
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

    # opencv的图像是BGR格式的，而我们需要是的RGB格式的，因此需要进行一个转换。
    rgb_small_frame = small_frame[:, :, ::-1]

    # Only process every other frame of video to save time
    if process_this_frame:
        # 根据encoding来判断是不是同一个人，是就输出true，不是为flase
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        face_names = []
        for face_encoding in face_encodings:
            # 默认为unknown
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"

            # if match[0]:
            #     name = "michong"
            # If a match was found in known_face_encodings, just use the first one.
            if True in matches:
                first_match_index = matches.index(True)
                name = known_face_names[first_match_index]
            face_names.append(name)

    process_this_frame = not process_this_frame

    # 将捕捉到的人脸显示出来
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        # Scale back up face locations since the frame we detected in was scaled to 1/4 size
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        # 矩形框
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        #加上标签
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

    # Display
    cv2.imshow('monitor', frame)

    # 按Q退出
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()


error: OpenCV(3.4.2) /io/opencv/modules/imgproc/src/resize.cpp:4044: error: (-215:Assertion failed) !ssize.empty() in function 'resize'


In [139]:
dft = pd.DataFrame()
                            

In [142]:
dft['工号'] = [1]

In [143]:
dft

,工号
0,1


In [144]:
dft['姓名'] =[' name']

In [145]:
dft

,工号,姓名
0,1,name


In [147]:
np.argmax([1,3,2,2,3,3,2,1])

1